In [1]:
import os
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from PIL import Image

Using TensorFlow backend.


In [2]:
train_dir = "../../../data_kaggle/train/train"

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

In [4]:
batch_size=64

train_generator = train_datagen.flow_from_directory(
        os.getcwd()+os.path.sep+train_dir,
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training') 

validation_generator = train_datagen.flow_from_directory(
        os.getcwd()+os.path.sep+train_dir,
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation') 

Found 3803 images belonging to 12 classes.
Found 947 images belonging to 12 classes.


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(10, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='relu'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 20)        5780      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 20)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 10)          1810      
_________________________________________________________________
flatten_1 (Flatten)  

In [6]:
n_epochs=20

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = n_epochs)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
59/59 [==============================] - 44s 748ms/step - loss: 4.3429 - acc: 0.1141 - val_loss: 4.2382 - val_acc: 0.1362
Epoch 2/20
59/59 [==============================] - 40s 677ms/step - loss: 4.2245 - acc: 0.1498 - val_loss: 4.0154 - val_acc: 0.2809
Epoch 3/20
59/59 [==============================] - 44s 754ms/step - loss: 4.0860 - acc: 0.2342 - val_loss: 4.0067 - val_acc: 0.3228
Epoch 4/20
59/59 [==============================] - 40s 686ms/step - loss: 4.1305 - acc: 0.2385 - val_loss: 3.8388 - val_acc: 0.2978
Epoch 5/20
59/59 [==============================] - 41s 688ms/step - loss: 4.0029 - acc: 0.3321 - val_loss: 5.1075 - val_acc: 0.2820
Epoch 6/20
59/59 [==============================] - 42s 710ms/step - loss: 3.9315 - acc: 0.3287 - val_loss: 3.7410 - val_acc: 0.3737
Epoch 7/20
59/59 [==============================] - 43s 721ms/step - loss: 3.8775 - acc: 0.3680 - val_loss: 4.0605 - val_acc: 0.2967
Epoch 8/20
59/59 [===

#### GENERATE KAGGLE RESULTS

In [7]:
test_dir = "../../../data_kaggle/test"
test_datagen = ImageDataGenerator(rescale=1./255)

print(os.getcwd()+os.path.sep+test_dir)
test_generator = test_datagen.flow_from_directory(
        os.getcwd()+os.path.sep+test_dir,
        target_size=(32, 32),
        color_mode="rgb",
        batch_size=1,
        class_mode=None,
        shuffle=False)
filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

C:\Users\ferran.marti.torrent\OneDrive - Accenture\19_project_kf\001_plant_seeding\ferdi\../../../data_kaggle/test
Found 794 images belonging to 1 classes.


In [8]:
import numpy as np
predicted_class_indices=np.argmax(predict,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"file":filenames,
                      "species":predictions})
results.to_csv("results.csv",index=False)